# AdventureWorks EDA with Python

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)

More examples are available on the Spark website: http://spark.apache.org/examples.html

PySpark API documentation: http://spark.apache.org/docs/latest/api/python/

## Author: Bryan Cafferky Copyright 08/26/2020

#### With Databricks, there's no need to import PySpark or create a Spark Context...

### Using RDDs with Python

Code adpated from https://spark.apache.org/docs/latest/rdd-programming-guide.html#basics
and from https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/

In [0]:
sc

In [0]:
spark

In [0]:
poem_rdd1 = sc.textFile("dbfs:/FileStore/tables/hamlet-1.txt")
lineLengths = poem_rdd1.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
print(totalLength)

In [0]:
my_rdd  = sc.parallelize([1, 2, 3, 4, 5])
my_rdd.count()

In [0]:
print(my_rdd)

In [0]:
poem_rdd1.getNumPartitions() 

In [0]:
poem_rdd1.take(5)

In [0]:
def transfunc(lines):
      lines = lines.lower()
      lines = lines.split()
      return lines
    
poem_rdd2 = poem_rdd1.map(transfunc)
poem_rdd2.take(5)

In [0]:
poem_rdd3 = poem_rdd1.flatMap(transfunc)
poem_rdd3.take(5)

In [0]:
poem_rdd3.distinct().take(5)

In [0]:
skipwords = ['to','the','of']
poem_rdd4 = poem_rdd3.filter(lambda x: x not in skipwords)
poem_rdd4.take(10)

In [0]:
poem_rdd5 = poem_rdd3.map(lambda x: (x,1))
poem_rdd5.take(5)

In [0]:
poem_rdd5_grouped = poem_rdd5.groupByKey()
print(list((j[0], list(j[1])) for j in poem_rdd5_grouped.take(5)))

In [0]:
poem_freq_of_words = poem_rdd5_grouped.mapValues(sum) \
  .map(lambda x: (x[1],x[0])).sortByKey(False)
poem_freq_of_words.take(10)

In [0]:
poem_rdd5.reduceByKey(lambda x,y: x+y) \
  .map(lambda x:(x[1],x[0])) \
  .sortByKey(False).take(10)

In [0]:
poem_rdd5_sampled = poem_rdd5.sample(False, 0.5, 42)
print(len(poem_rdd5.collect()),len(poem_rdd5_sampled.collect()))

In [0]:
numbers_rdd = sc.parallelize(range(1,500))
numbers_rdd.max(),numbers_rdd.min(), numbers_rdd.sum(),numbers_rdd.variance(),numbers_rdd.stdev()

### Loading and joining two rdds loaded from JSON files.

In [0]:
ssales_rdd = spark.read.json("dbfs:/FileStore/tables/storesales.json").rdd
ssales_rdd2 = ssales_rdd.map(lambda y: (y[0], y[1]))
print('Key lookup:  ', ssales_rdd2.filter(lambda x: x[0] == 'MA').collect())
print('rdd elements: \n', ssales_rdd2.take(8))

In [0]:
squotas_rdd = spark.read.json("dbfs:/FileStore/tables/storequota.json").rdd
squotas_rdd2 = squotas_rdd.map(lambda y: (y[0], y[1]))
print('Key lookup: ', squotas_rdd2.filter(lambda x: x[0] == 'RI').collect())
print('rdd elements: \n', squotas_rdd2.take(8))

In [0]:
storejoined_rdd = ssales_rdd2.join(squotas_rdd2)
print('rdd elements after join: \n', squotas_rdd2.take(8))
print('Key lookup: ', storejoined_rdd.filter(lambda x: x[0] == 'ME').collect())

## Now let's use Spark SQL

In [0]:
spark.sql('use aw')
spdf_salesinfo = spark.sql('select * from t_salesinfo').dropna()

In [0]:
getcustgeoqry = '''
select CustomerKey, CountryRegionCode, EnglishCountryRegionName as CountryName, 
StateProvinceCode, StateProvinceName, City, PostalCode
from  t_customerinfo      c
inner join  dimgeography  g
on (c.geographykey = g.geographykey) 
where countryregioncode = 'US'
'''

In [0]:
spdf_custgeo_us = spark.sql(getcustgeoqry)
display(spdf_custgeo_us.limit(3))

In [0]:
display(spdf_salesinfo.join(spdf_custgeo_us, spdf_salesinfo.CustomerKey == spdf_custgeo_us.CustomerKey, 'inner').select('*').take(3))

In [0]:
spdf_salesinfo_us = spdf_salesinfo.join(
spdf_custgeo_us, spdf_salesinfo.CustomerKey == spdf_custgeo_us.CustomerKey, 'inner').select('*')

display(spdf_salesinfo_us.collect())

In [0]:
display(spdf_salesinfo_us.describe(['SalesAmount','Salary']))

In [0]:
display(spdf_salesinfo_us)

In [0]:
display(spdf_salesinfo_us)

In [0]:
display(spdf_salesinfo_us)

In [0]:
display(spdf_salesinfo_us)

In [0]:
display(spdf_salesinfo_us.groupby('CommuteDistance').
        agg({'SalesAmount': 'sum'}).
        orderBy('sum(SalesAmount)'))

In [0]:
display(spdf_salesinfo_us.groupby('CommuteDistance').
        agg({'SalesAmount': 'sum', '*': 'count'}).
        orderBy('count(1)'))

In [0]:
display(spdf_salesinfo_us.groupBy(['AgeBand']).agg({'SalesAmount': 'sum'}).collect())

In [0]:
spdf_salesinfo_us.createOrReplaceTempView('vsalesinfo_us')

In [0]:
# Take a sample so you don't bring too much data back to the cluster driver.

ppdf_salesinfo_us = spdf_salesinfo_us.sample(False, 0.5, 42).toPandas()

In [0]:
spdf_salesinfo_us.printSchema()

In [0]:
# Because you created the PyPark df using 'select(*), you got CustomerKey two times so just drop that column and then
# take a sample so you don't bring too much data back to the cluster driver.

ppdf_salesinfo_us = spdf_salesinfo_us. \
   drop('CustomerKey'). \
   sample(False, 0.5, 42).toPandas()

In [0]:
display(ppdf_salesinfo_us.head(3))

In [0]:
import pandas as pd

ppdf_salesbygender = ppdf_salesinfo_us.groupby(['Gender']).agg({'SalesAmount': 'sum'})
ppdf_salesbygender.plot(kind='bar', subplots=True, color='purple')
display()

In [0]:
ppdf_salesinfo_us.groupby('Gender')['SalesAmount'].sum()

In [0]:
# import some more functions
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import avg
from pyspark.sql.functions import max, min
from pyspark.sql.functions import stddev

# Perform aggregations on the DataFrame
agg = spdf_salesinfo_us.agg(
    avg(spdf_salesinfo_us.SalesAmount).alias("avg_sales"), 
    stddev(spdf_salesinfo_us.SalesAmount).alias("stddev_sales"),
    min(spdf_salesinfo_us.SalesAmount).alias("min_sales"),
    max(spdf_salesinfo_us.SalesAmount).alias("max_sales"),
    countDistinct(spdf_salesinfo_us.ProductKey).alias("distinct_products"), 
    countDistinct(spdf_salesinfo_us['c.CustomerKey']).alias('distinct_customers')
)

# Convert the results to Pandas DataFrame
display(agg.toPandas())

In [0]:
%r
library(SparkR)

display(sql("select * from vsalesinfo_us"))

In [0]:
spdf_salesinfo_us.drop('CustomerKey').createOrReplaceTempView('vsalesinfo_us')

In [0]:
%sql 

DROP TABLE IF EXISTS t_salesinfo_us

In [0]:
%sql

CREATE TABLE t_salesinfo_us as 
SELECT * FROM vsalesinfo_us

In [0]:
%sql 

SELECT MonthNumberOfYear, sum(SalesAmount) 
FROM t_salesinfo_us 
GROUP BY MonthNumberOfYear 
ORDER BY MonthNumberOfYear

In [0]:
# Use the Spark CSV datasource with options specifying:
# - First line of file is a header
# - Automatically infer the schema of the data

spdf_sales = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/tables/FactInternetSales.csv")
spdf_factreason = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/tables/FactInternetSalesReason.csv")
spdf_dimreason = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/tables/DimSalesReason.csv")


#sales.cache() # Cache data for faster reuse
spdf_sales = spdf_sales.dropna() # drop rows with missing values

In [0]:
spdf_sales.createOrReplaceTempView('vsalestemp')
spdf_factreason.createOrReplaceTempView('vfactreasontemp')
spdf_dimreason.createOrReplaceTempView('vdimreasontemp')

In [0]:
%sql

SELECT sum(SalesAmount) as TotalSales, ifnull(SalesReasonName, 'None') as SalesReason
FROM vsalestemp              s
LEFT JOIN vfactreasontemp   fr
ON (s.SalesOrderNumber = fr.SalesOrderNumber) and (s.SalesOrderLineNumber = fr.SalesOrderLineNumber)
LEFT JOIN vdimreasontemp    dr
ON (fr.SalesReasonKey = dr.SalesReasonKey)
GROUP BY ifnull(SalesReasonName, 'None')

## PySpark User Defined Functions

In [0]:
%sql USE aw

In [0]:
%sql describe table t_salesinfo_us

In [0]:
%sql

SELECT instr(Model, 'Mount'), substring(Model, 1,2), translate(Model, 'o', 'X'), Model 
FROM aw.t_salesinfo_us LIMIT 2;

In [0]:
spdf_salesinfo_us = spark.sql("select * from aw.t_salesinfo_us limit 3")

In [0]:
from pyspark.sql.functions import instr, translate, substring

display(
spdf_salesinfo_us.select(instr("Model", "Mount"), substring("Model",1,2), translate('Model', 'o', 'X'))
)

In [0]:
from pyspark.sql.types import DoubleType

def margin_precent_type(productcost, saleamt):
  return (saleamt - productcost) / saleamt

spark.udf.register("margin_percent", margin_precent_type, DoubleType())

In [0]:
%sql 

SELECT margin_percent(TotalProductCost, SalesAmount) as gross_margin, TotalProductCost, SalesAmount 
FROM  aw.t_salesinfo_us LIMIT 2;

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

margin_percent_udf = udf(margin_precent_type, DoubleType())

In [0]:
display(spdf_salesinfo_us.select("SalesAmount", "TotalProductCost", 
                                 margin_percent_udf("TotalProductCost",
                                 "SalesAmount").alias("MarginPercent")))

In [0]:
from pyspark.sql.functions import udf

@udf("double")
def margin_precent_type_udf2(productcost, saleamt):
  return (saleamt - productcost) / saleamt

In [0]:
display(spdf_salesinfo_us.select("SalesAmount", "TotalProductCost", 
                                 margin_precent_type_udf2("TotalProductCost", 
                                 "SalesAmount").alias("MarginPercent")))

In [0]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

## Pandas Dataframes

Good blog at: https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html

## Using an iterator and passing more than one column, i.e. a tuple. 

See link for more info: https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/udf-python-pandas

In [0]:
# When the UDF is called with more than one column,
# the input to the underlying function is an iterator of pd.Series tuple.

import pandas as pd

from pyspark.sql.functions import col, pandas_udf
        
@pandas_udf("double", PandasUDFType.SCALAR_ITER)        
def margin_precent_udf(batch_iter):
    for saleamt, productcost in batch_iter:
        yield (saleamt - productcost) / saleamt     
        
spdf_salesinfo_us.select("SalesAmount", "TotalProductCost", 
                         margin_precent_udf(col("SalesAmount"), col("TotalProductCost"))).show(3)        

In [0]:
spdf_salesinfo_us2 = spark.sql("select * from aw.t_salesinfo_us")

Original Link: https://stackoverflow.com/questions/40006395/applying-udfs-on-groupeddata-in-pyspark-with-functioning-python-example

In [0]:
from pyspark.sql.functions import pandas_udf,PandasUDFType

from pyspark.sql.types import *

import pandas as pd

# Get a set of data to process...
spdf_salesinfo_us2 = spark.sql("select * from aw.t_salesinfo_us")

schema = StructType([
    StructField("SalesTerritoryKey", IntegerType()),
    StructField("avg_productcost", DoubleType()),
    StructField("avg_saleamount", DoubleType()),
    StructField("avg_profitmargin", DoubleType())
])

@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)
def avg_profit_margin(df):
    gr = df['SalesTerritoryKey'].iloc[0]
    pc = df.TotalProductCost.mean()
    sa = df.SalesAmount.mean()
    gm = (df.SalesAmount.mean() - df.TotalProductCost.mean()) / df.TotalProductCost.mean()
    return pd.DataFrame([[gr]+[pc]+[sa]+[gm]])

display(spdf_salesinfo_us2.groupby("SalesTerritoryKey"). \
        apply(avg_profit_margin).orderBy('SalesTerritoryKey'))

See https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.apply

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import Window

df = spark.createDataFrame(
    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],
    ("id", "v"))

@pandas_udf("double", PandasUDFType.GROUPED_AGG)
def profit_margin_udf(saleamt, productcost):
    return (saleamt.mean() - productcost.mean()) / saleamt.mean() 

w = Window \
    .partitionBy('SalesTerritoryKey') \
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

spdf_profit = spdf_salesinfo_us2.withColumn('profit_margin', 
                                            profit_margin_udf(spdf_salesinfo_us2['SalesAmount'], 
                                            spdf_salesinfo_us2['TotalProductCost']).over(w))

display(spdf_profit.select('SalesTerritoryKey', 'SalesAmount', 'TotalProductCost', 'profit_margin'))